#### Directories etc

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
import random
from tqdm import tqdm
from Comparative_Analysis import Utilities as util
import random
from joblib import Parallel, delayed
from Comparative_Analysis import Blast_Functions as blastfn
import time
import os
import copy
from scipy.stats import chi2, chi2_contingency
from Comparative_Analysis import ORF_Functions as orffn
from Comparative_Analysis import Alignment as alignfn
from random import sample
from Bio.Align.Applications import MuscleCommandline
import subprocess
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#project_dir = 'F:/Project_Data/Intergenic_Region_Comparative_Analysis'
project_dir = 'F:/Project_Data/Intergenic_Region_Comparative_Analysis_Downstream'
seq_dir_1 = 'F:/Datasets/NCBI_Refseq_Mycobacteriaceae_All_Levels/ncbi_dataset/data'
seq_dir_2 = 'F:/Datasets/NCBI_Refseq_Actinobacteria_All_Levels/data'
sequence_dirs_1 = util.list_dirs(seq_dir_1)
sequence_dirs_2 = util.list_dirs(seq_dir_2)
tb_species = 'NC_000962.3' 
tb_annotation_dirname = 'GCF_000195955.2'
num_cores = 16
core_numbers = list(range(1, num_cores+1))
muscle_exe = 'C:/Users/nicho/Muscle/muscle3.8.31_i86win32.exe'
full_build = False

In [3]:
cortes_dict = {}
utr_category_dict = {}
cortes_df = pd.read_csv('F:/Datasets/Data_From_Publications/Cortes_TSS.csv')
for i, r in cortes_df.iterrows():
    cortes_dict[r['RvNumber']] = r['Leaderless']
    utr_category_dict[r['RvNumber']] = r['UTR Category']

In [4]:
cds_info_dict = {}
for genome_record in SeqIO.parse(seq_dir_1 + '/' + tb_annotation_dirname + '/genomic.gbff', "genbank"):
    for feature in genome_record.features:
        a = feature.qualifiers
        if feature.type == 'CDS' and a.get("translation") != None:
            locus_tag = a.get("locus_tag")[0]
            product = a.get("product")[0]
            cds_info_dict[locus_tag] = (feature.location.start, feature.location.end, feature.location.strand)

#####  Find potential structural regions

In [7]:
region_list = []
seq_ids = util.list_dirs(project_dir)
for id in seq_ids:
    if os.path.exists(project_dir + '/' + str(id) + '/rscape_output_2.cov'):
        alignment = util.read_fasta_to_array(project_dir + '/' + str(id) + '/upstream_region_'+str(id) + '_alignment_2.fasta')
        num_sequences = len(alignment[0])
        num_characters = len(alignment[1][0])
  
        with open(project_dir + '/'+str(id) +  '/rscape_output_2.cov', 'r') as f:  
            pair_items = []
            num_pairs = 0
            e_values = []

            for l in f:
                if (not ('#' in l)):
                    a = l.split()
                    if len(a) > 6 and abs(int(a[2])-int(a[1])) > 5 and abs(int(a[1])-num_characters) > 10 and abs(int(a[2])-num_characters) > 10:    # Exclude covarying pairs less than 3 bp apart)
                        e_values.append(float(a[4]))
                        num_pairs+=1
                        pair_items.append(int(a[2]))
                        pair_items.append(int(a[2]))
        if len(e_values) > 0:
            tot = sum([math.log(x) for x  in e_values])
            num = len(e_values)
            combined_e_value = 1-chi2.cdf(-2*tot, 2*num)
        else:
            combined_e_value = 999
        if combined_e_value < 1e-5:
            region_list.append((id, num_pairs, len(set(pair_items)), combined_e_value))
region_list.sort(key = lambda x: x[0])

[(x, cds_info_dict[x[0]]) for x in region_list if x[2] > 8]


[(('Rv0071', 15, 13, 0.0), (ExactPosition(79485), ExactPosition(80193), 1)),
 (('Rv0129c', 14, 9, 0.0), (ExactPosition(156577), ExactPosition(157600), -1)),
 (('Rv0189c', 18, 9, 0.0), (ExactPosition(219995), ExactPosition(221723), -1)),
 (('Rv2511', 84, 21, 0.0),
  (ExactPosition(2827156), ExactPosition(2827804), 1)),
 (('Rv3661', 20, 13, 0.0),
  (ExactPosition(4099646), ExactPosition(4100510), 1)),
 (('Rv3684', 586, 55, 0.0),
  (ExactPosition(4125438), ExactPosition(4126479), 1)),
 (('Rv3752c', 36, 19, 0.0),
  (ExactPosition(4199246), ExactPosition(4199705), -1))]

In [8]:
pd.DataFrame([x for x in region_list if x[2] > 8]).to_csv(project_dir +'/high_covarying_downstream_regions.csv')